In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# load env file all variable
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['HUGGINGFACE_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage



In [2]:
llm = ChatGroq(model='llama3-70b-8192')

In [3]:
llm.invoke([HumanMessage(content="hi, i am hannan, i am a machine learning engineer")])

AIMessage(content="Hi Hannan! It's nice to meet you. As a machine learning engineer, you must be working on some exciting projects. What's your current focus? Are you working on a particular domain like computer vision, NLP, or something else?\n\nAlso, what kind of problems do you enjoy solving? Are you interested in the theoretical aspects of machine learning or do you prefer working on practical applications and deploying models to production?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 23, 'total_tokens': 109, 'completion_time': 0.330678164, 'prompt_time': 0.000334426, 'queue_time': 0.223043519, 'total_time': 0.33101259}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--ef26e539-d153-4e5a-aee2-58d139596d3c-0', usage_metadata={'input_tokens': 23, 'output_tokens': 86, 'total_tokens': 109})

In [4]:
llm.invoke([
    HumanMessage(content="hi, i am hannan, i am a machine learning engineer"),
    AIMessage(content="Hi Hannan! It's great to meet you. As a machine learning engineer, you must be working on some exciting projects. What kind of projects are you currently involved in? Are you working on any specific domains like NLP, Computer Vision, or something else?"),
    HumanMessage(content="Hey, whats my name and what do it do?")
])

AIMessage(content="Your name is Hannan, and you're a Machine Learning Engineer!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 98, 'total_tokens': 113, 'completion_time': 0.053683469, 'prompt_time': 0.00378679, 'queue_time': 0.218991315, 'total_time': 0.057470259}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--11e582ce-3c03-49d9-81c7-40b04a9285ce-0', usage_metadata={'input_tokens': 98, 'output_tokens': 15, 'total_tokens': 113})

#### Message history

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

In [6]:
store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [7]:
config = { "configurable": {"session_id": "chat1"}  }

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="hi, i am hannan, i am a machine learning engineer")],
    config=config
)

In [9]:
response.content

'Hi Hannan, nice to meet you! As a machine learning engineer, you must be working on some exciting projects. What kind of projects are you currently working on? Are you more into NLP, Computer Vision, or something else?'

In [10]:
response2 = with_message_history.invoke(
    [HumanMessage(content='what is my name')],
    config=config
)

In [11]:
response2.content

"I remember! Your name is Hannan, and you're a machine learning engineer."

In [12]:
# now lets create new session id
config1 = { "configurable": {"session_id": "chat2"}}

res = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)

res.content

"I'm sorry! I don't know your name. I'm a large language model, I don't have the ability to know personal information about you. You didn't tell me your name, and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!"

In [13]:

res = with_message_history.invoke(
    [HumanMessage(content="my name is jhon")],
    config=config1
)

res.content

"Nice to meet you, Jhon! It's great that you shared your name with me. I'll make sure to address each other properly from now on. How's your day going so far, Jhon?"

In [14]:

res = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)

res.content

'I remember! Your name is Jhon.'

#### Manage the conversation history

In [15]:
from langchain_core.messages import SystemMessage, trim_messages

In [60]:
trimmer = trim_messages(
    max_tokens=45,
    token_counter=llm,
    strategy="last", # The tokens from the beginning will be kept. and `last` => The tokens from the ending will be kept.
    allow_partial=True,
    start_on="human",
    include_system = True
)

In [122]:
messages = [
    SystemMessage(content="you are a good ai assistent"),
    HumanMessage(content="hello, i am hannan"),
    AIMessage(content="hi, i am AI"),
    HumanMessage(content="I like programming"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 5 + 5"),
    AIMessage(content="10"),
    HumanMessage(content="Having fun!"),
    AIMessage(content="yes"),
    HumanMessage(content="Thank you very much"),
    AIMessage(content="you are most welcome"),
]
trimmer.invoke(messages)

[SystemMessage(content='you are a good ai assistent', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 5 + 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='10', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thank you very much', additional_kwargs={}, response_metadata={}),
 AIMessage(content='you are most welcome', additional_kwargs={}, response_metadata={})]

#### Prompt template

In [64]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [86]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you ara en expert ai assistent. provide me answer based on questions"),
        MessagesPlaceholder(variable_name="chat_history")
    ]
)

chain = prompt|llm


In [87]:
chain.invoke({'chat_history': [HumanMessage(content="hi! my name is hannan")]})


AIMessage(content="Hi Hannan! It's nice to meet you. I'm your AI assistant, here to provide you with helpful and accurate answers to your questions. What's on your mind today? Do you have a specific question or topic you'd like to discuss?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 36, 'total_tokens': 88, 'completion_time': 0.148571429, 'prompt_time': 0.000771921, 'queue_time': 0.21968833299999999, 'total_time': 0.14934335}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--089d7a8b-a5c0-4ead-baec-9c6f9e940e73-0', usage_metadata={'input_tokens': 36, 'output_tokens': 52, 'total_tokens': 88})

In [88]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [90]:
config = {"configurable": {"session_id": "chat4"}}

In [92]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Hannan")],
    config=config
)

response.content

"Nice to meet you, Hannan! It's great to have you interacting with me. Is there anything specific you'd like to talk about, ask, or discuss? I'm all ears (or rather, all text)!"

In [93]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

response.content

'I remember! Your name is Hannan.'

In [97]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you ara en expert ai assistent. Answer all questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="chat_history")
    ]
)
chain = prompt | llm

In [99]:
result = chain.invoke({
    "chat_history": [HumanMessage(content="Hi, my name is Hannan")],
    "language": "hindi"
})
result.content

'नमस्ते हनन! मैं एक एक्सपर्ट एआई असिस्टेंट हूँ। मैं आपकी सभी प्रश्नों का उत्तर देने का प्रयास करूँगा। क्या आप मुझसे कुछ पूछना चाहते हैं?'

In [100]:
result = chain.invoke({
    "chat_history": [HumanMessage(content="Hi, my name is Hannan")],
    "language": "bangla"
})
result.content

'নমস্কার হান্নান) ! আমি আপনাকে সহায়তা করতে পারি । আপনি কি আমাকে কিছু জিজ্ঞাসা করতে চান?'

In [ ]:
result = chain.invoke({
    "chat_history": [HumanMessage(content="Hi, my name is Hannan")],
    "language": "japanes"
})
result.content

In [101]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="chat_history"
)

In [107]:
config4 = {"configurable": {"session_id": "chat5"}}

In [108]:
response = with_message_history.invoke(
    {"chat_history": [HumanMessage(content="my name is Hannan")], "language": "bangla"},
    config=config4
)

In [109]:
response.content

'ালিকুম! আমি খুশি যে আপনার নাম হান্নান। আপনার সাথে কথা বলতে আমি খুব খুশি। কী বিষয়ে আপনি আমার সাহায্য চান?'

In [110]:
response = with_message_history.invoke(
    {"chat_history": [HumanMessage(content="What is my name")], "language": "bangla"},
    config=config4
)

In [111]:
response.content

'আপনার নাম হান্নান!'

In [112]:
response = with_message_history.invoke(
    {"chat_history": [HumanMessage(content="i love computer programming")], "language": "bangla"},
    config=config4
)
response.content

'আহা! কম্পিউটার প্রোগ্রামিং আপনার পছন্দ! আপ্রোগ্রামিং ভাষা কোনটা আপনার সবচেয়ে পছন্দ? জাভা, পাইথন, সি++, না কি অন্য কিছু?'

In [113]:
response = with_message_history.invoke(
    {"chat_history": [HumanMessage(content="i love python"), HumanMessage(content="i am a machine learning engineer")], "language": "bangla"},
    config=config4
)
response.content

'আহা! মেশিন লার্নিংজ ইঞ্জিনিয়ার আপনি! আপনি কোন ধরনের প্রজেক্টে কাজ করছেন? কম্পিউটার ভিশন, ন্যাচারাল ল্যাঙ্গুইজ প্রসেসিং, বা অন্য কিছু?'

In [114]:
response = with_message_history.invoke(
    {"chat_history": [HumanMessage(content="which programming language i love")], "language": "bangla"},
    config=config4
)
response.content

'আহা! আপনি পাইথন ভাষা পছন্দ করেন!'

In [115]:
response = with_message_history.invoke(
    {"chat_history": [HumanMessage(content="what is my profession?")], "language": "bangla"},
    config=config4
)
response.content

'আহা! আপনি একজন মেশিন লার্নিংজ ইঞ্জিনিয়ার!'